## Programming Drill 3.3.2 
### Modify your program from Programming drill 3.2.2 so that you allow transitions from the many slits to the many measuring devices to be complex numbers. Your program should identify where there are interference phenomena. 

In [2]:
#!/usr/bin/env python3
"""
NumPy version of Programming Drill 3.3.2
Multislit experiment with complex amplitudes and interference detection.

Usage:
 - Enter integers for s (slits) and m (targets)
 - Enter gun->slits amplitudes as space-separated expressions, e.g.:
     1/sqrt(2) 1/sqrt(2)
   or:
     0.70710678 0.70710678
 - Enter slit->targets amplitudes likewise; use sqrt(...) if you want.
 - The parser accepts expressions with sqrt(...) and basic arithmetic.
"""

import numpy as np
import math
from typing import List

EPS = 1e-12

def parse_expr_to_complex(token: str) -> complex:
    """
    Parse expression strings like '1', '1/2', '1/sqrt(3)', '0.7071+0.7071j', '-0.3j'
    We allow: + - * / parentheses, sqrt(x). We evaluate using a safe local mapping.
    """
    s = token.strip()
    # allow explicit 'j' imaginary inputs and Python complex literals
    try:
        # try direct python complex parsing first
        return complex(s)
    except Exception:
        pass
    # replace sqrt(...) with math.sqrt(...)
    safe = s.replace('sqrt', 'math.sqrt')
    # disallow suspicious names by providing limited globals
    allowed_globals = {"__builtins__": None, "math": math}
    try:
        val = eval(safe, allowed_globals, {})
        return complex(val)
    except Exception as e:
        raise ValueError(f"Cannot parse '{token}': {e}")

def read_int(prompt: str) -> int:
    while True:
        try:
            v = int(input(prompt).strip())
            if v <= 0:
                print("Please enter a positive integer.")
                continue
            return v
        except Exception:
            print("Enter a valid integer.")

def read_complex_vector_numpy(n: int, prompt: str) -> np.ndarray:
    while True:
        line = input(prompt).strip()
        parts = line.split()
        if len(parts) != n:
            print(f"Expected {n} entries; got {len(parts)}. Try again.")
            continue
        try:
            vals = [parse_expr_to_complex(p) for p in parts]
            return np.array(vals, dtype=np.complex128)
        except ValueError as e:
            print(e)

def build_B_numpy(s: int, m: int,
                  gun_to_slits: np.ndarray,
                  slit_to_targets: np.ndarray) -> np.ndarray:
    """
    gun_to_slits: shape (s,)
    slit_to_targets: shape (s, m)  (each row j gives amplitudes from slit j -> target t)
    Returns B of shape (1+s+m, 1+s+m) with complex dtype
    Rows = next position, Cols = current position
    """
    n = 1 + s + m
    B = np.zeros((n, n), dtype=np.complex128)

    # gun column -> slits
    for i in range(s):
        B[1 + i, 0] = gun_to_slits[i]

    # slit columns -> targets
    for j in range(s):
        col = 1 + j
        # slit_to_targets[j] length m -> rows 1+s .. 1+s+m-1
        B[1 + s : 1 + s + m, col] = slit_to_targets[j, :]

    # absorbing targets
    for t in range(m):
        idx = 1 + s + t
        B[idx, idx] = 1.0 + 0j

    return B

def pretty_complex(z: complex, prec=6) -> str:
    r = round(z.real, prec)
    im = round(z.imag, prec)
    if abs(im) < 10**(-prec):
        return f"{r}"
    if abs(r) < 10**(-prec):
        return f"{im}j"
    sign = '+' if im >= 0 else '-'
    return f"{r}{sign}{abs(im)}j"

def main():
    print("NumPy Multislit (complex amplitudes) — interference detection\n")
    s = read_int("Enter number of slits (s): ")
    m = read_int("Enter number of targets (m): ")

    gun = read_complex_vector_numpy(s, f"Enter gun -> slits amplitudes ({s} values): ")
    slit_to_targets = np.zeros((s, m), dtype=np.complex128)
    for j in range(s):
        slit_to_targets[j, :] = read_complex_vector_numpy(m, f"Enter slit {j+1} -> {m} targets amplitudes: ")

    B = build_B_numpy(s, m, gun, slit_to_targets)
    n = 1 + s + m

    print("\nB matrix (rows=next, cols=current) — showing a few decimals:")
    for row in range(n):
        print("  " + "  ".join(pretty_complex(B[row, col]) for col in range(n)))

    # compute B^2 quickly
    B2 = B @ B
    print("\nB^2 (first few decimals shown):")
    for row in range(n):
        print("  " + "  ".join(pretty_complex(B2[row, col]) for col in range(n)))

    # initial amplitude at gun
    v0 = np.zeros(n, dtype=np.complex128)
    v0[0] = 1.0 + 0j

    # amplitude after two clicks (targets are at indices 1+s ... 1+s+m-1)
    v2_all = B2 @ v0
    target_idx = np.arange(1 + s, 1 + s + m)
    amps_all_targets = v2_all[target_idx]
    probs_all_targets = np.abs(amps_all_targets)**2

    print("\nTargets (index, amplitude, probability):")
    for t in range(m):
        ai = amps_all_targets[t]
        pi = probs_all_targets[t]
        print(f" target {t+1} (vertex {target_idx[t]}): amp={pretty_complex(ai)}, prob={pi:.12g}")

    # Single-slit runs (incoherent sum)
    probs_each = []
    for j in range(s):
        gun_j = np.zeros(s, dtype=np.complex128)
        gun_j[j] = gun[j]
        B_j = build_B_numpy(s, m, gun_j, slit_to_targets)
        B_j2 = B_j @ B_j
        v2_j = B_j2 @ v0
        amps_j = v2_j[target_idx]
        probs_j = np.abs(amps_j)**2
        probs_each.append(probs_j)

    probs_each = np.array(probs_each)  # shape (s, m)
    probs_incoherent = probs_each.sum(axis=0)

    print("\nIncoherent sum (sum of single-slit probabilities) per target:")
    for t in range(m):
        print(f" target {t+1}: P_incoherent = {probs_incoherent[t]:.12g}")

    # Compare coherent vs incoherent => interference detection
    print("\nComparison and interference detection:")
    interference = False
    for t in range(m):
        p_all = probs_all_targets[t]
        p_inc = probs_incoherent[t]
        diff = p_all - p_inc
        if abs(diff) > 1e-9:
            interference = True
            kind = "constructive" if diff > 0 else "destructive"
            print(f" target {t+1}: P_all = {p_all:.12g}, ΣP_single = {p_inc:.12g}, Δ = {diff:.12g}  --> {kind} interference")
        else:
            print(f" target {t+1}: P_all ≈ ΣP_single (Δ ≈ 0) — no interference detected")

    tot_all = probs_all_targets.sum()
    tot_inc = probs_incoherent.sum()
    print(f"\nTotal prob on targets (coherent): {tot_all:.12g}")
    print(f"Total prob on targets (incoherent sum): {tot_inc:.12g}")

    if interference:
        print("\nInterference detected at one or more targets.")
    else:
        print("\nNo interference detected.")

if __name__ == "__main__":
    main()


NumPy Multislit (complex amplitudes) — interference detection


B matrix (rows=next, cols=current) — showing a few decimals:
  0.0  0.0  0.0  0.0  0.0  0.0
  0.707107  0.0  0.0  0.0  0.0  0.0
  0.707107  0.0  0.0  0.0  0.0  0.0
  0.0  0.57735  0.57735  1.0  0.0  0.0
  0.0  0.57735  -0.57735  0.0  1.0  0.0
  0.0  0.57735  0.57735  0.0  0.0  1.0

B^2 (first few decimals shown):
  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0
  0.816497  0.57735  0.57735  1.0  0.0  0.0
  -0.0  0.57735  -0.57735  0.0  1.0  0.0
  0.816497  0.57735  0.57735  0.0  0.0  1.0

Targets (index, amplitude, probability):
 target 1 (vertex 3): amp=0.816497, prob=0.666666666667
 target 2 (vertex 4): amp=-0.0, prob=3.16704479712e-34
 target 3 (vertex 5): amp=0.816497, prob=0.666666666667

Incoherent sum (sum of single-slit probabilities) per target:
 target 1: P_incoherent = 0.333333333333
 target 2: P_incoherent = 0.333333333333
 target 3: P_incoherent = 0.333333333333

Com